In [1]:
import sys
import os

openagent_dir = os.path.dirname(os.path.abspath(""))
sys.path.append(openagent_dir)

In [2]:
from openagent import compiler
from openagent.agent.completion import TextCompletionAgent
from openagent.agent.chat import ChatAgent

from openagent.llms._openai import OpenAI
from openagent.vectorstores.chroma import Chroma
from openagent.text_splitter import CharacterTextSplitter
from openagent.knowledgebase.document_loaders.file.base import SimpleDirectoryReader
from openagent.knowledgebase.doc_loader import document_loader

from openagent.knowledgebase.base import SimpleKnowledgeBase
from openagent.vectorstores.embeddings.openai import OpenAIEmbeddings

# load .env file in format :
# OPENAI_API_KEY=sk-xxx
from dotenv import load_dotenv
load_dotenv()

llmcc = OpenAI(
    model="gpt-3.5-turbo"
)
llmtc = OpenAI(
    model="text-davinci-003"
)

SimpleDirectoryReader = document_loader('file_directory')

reader = SimpleDirectoryReader('test_data/meteoric')
raw_docs = reader.load_data()

splitter = CharacterTextSplitter(chunk_overlap=40, chunk_size=1024)

openai_embeddings = OpenAIEmbeddings()
chroma = Chroma(embedding_function=openai_embeddings)    

knowledge = SimpleKnowledgeBase(raw_data=raw_docs, data_transformer=splitter, vector_store=chroma)

c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# chat agent testing with knowledge

chat_template = '''
            {{#user~}}
            You will use this FORMAT only to answer user's QUERY
            FORMAT: {{format}}
            QUERY: {{input}}

            Use the below knowledge to answer QUERY in given FORMAT:-
            {{RETRIEVED_KNOWLEDGE}}
            {{~/user}}
                                        
            {{#assistant~}}
            Yes, I will tell you about with that
            {{~/assistant}}
        
            {{#user~}}
            Yes, tell me
            {{~/user}}
                                        
            {{#assistant~}}
            {{gen 'response' temperature=0 max_tokens=300}}
            {{~/assistant}}'''


agent = ChatAgent(
    llm = llmcc,
    prompt_template = chat_template,
    input_variables = {"knowledge_variable": 'input', "extras": 'format'},
    knowledgebase = knowledge,
    output_key='response'
    )

out = agent.run(input = "What was the first thing Alex said to Xarnak?", format = "JSON")

print(out)

The first thing Alex said to Xarnak was, "I am no longer the scared child you remember, Xarnak. I am the protector of this planet, and I won't let you harm it."


In [ ]:
agent.export_agent_config("test-agent-config.yaml")

In [7]:
agent.knowledgebase.vector_store._embedding_function

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='', openai_organization='', allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None)

In [6]:
# formatted output with text completion model without knowledge base

text_template = '''
City: "{{city}}"
Average temperature (C): "{{gen 'avg_temp'}}"
Population: "{{population}}"
Famous for (in 10 words): {{gen "famous_for" temperature=0.7 max_tokens=30}}'''

agent = TextCompletionAgent(
    llm=llmtc,
    prompt_template = text_template,
    return_complete=True,
    input_variables={'extras': ['city', 'population']}
    )

out = agent.run(city = 'New Delhi', population = "2 crore")

print(out)


City: "New Delhi"
Average temperature (C): "25.5""
Population: "2 crore"
Famous for (in 10 words):  "Historical monuments, cuisine, markets"


In [2]:
# QnA with text completion model on knowledge base

text_template = '''
Here is a part of a movie script:
{{RETRIEVED_KNOWLEDGE}}

Question: {{question}}
Answer: {{gen 'answer'}}
'''

agent = TextCompletionAgent(
    llm = llmtc,
    prompt_template = text_template,
    input_variables = {"knowledge_variable": 'question'},
    knowledgebase = knowledge,
    output_key='answer'
    )

out = agent.run(question = "What is the name of the hero in the movie?")

print(out)

In [5]:


# text completion with text completion model

text_template = '''
Here is one part of a movie script:
{{RETRIEVED_KNOWLEDGE}}

{{input}} {{gen 'complete'}}
'''

agent = TextCompletionAgent(
    llm = llmtc,
    prompt_template = text_template,
    input_variables = {"knowledge_variable": 'input'},
    knowledgebase = knowledge,
    output_key='complete'
    )

out = agent.run(input = "The hero of this movie was so ")
print(out)


powerful that they were able to unite the world under the banner of hope, compassion, and solidarity.
